# \**The Struggle\** is Real - A Quick Stab in the Dark at the Cultural-Commerical Roots of NYC's Current Chaos

## 0: Introduction

#### New York City is having lots of trouble. 

#### Its mayor and the governor of New York state have been using their authority in clearly unfair ways, like explicitly choosing to allow some New Yorkers to use their most basic rights--First Amendment rights--based on the political preferences they share as hypocritical liberals--even after a federal court ordered them to stop unfairly enforcing the most basic, important law in the country.

#### Recent riots in the city haven't been met with enough police action to prevent gross amounts of property damage and personal injury; the mayor wants to decrease funding for the city's police; and even before the recent riots, crime had increased in the city, following the state changing bail laws, which allowed arrested suspects a greater chance for pre-trial freedom.  

#### Is the problem in New York a clash of classes?

#### Politicians, economists and concerned NYers might do well look at the cultural-commercial environment of Manhattan, New York City's wealthiest borough, to see if any fundamental division can be derived from the types of venues frequented by its inhabitants.

## 1: Data

#### To help judge the potential divide, I'll use data as follows:

#### (1) I'll gather geospatial neighborhood data for Manhattan and 
#### (2) Foursquare consumer-provided data to 'explore', via the Foresquare function of the same name, Manhattan neighborhoods and their venues. 
#### (3) To come up with a mathematical guess about how alike or different neighborhoods across Manhattan might be, I'll use the popular and powerful *k*-means clustering algorithm. 
#### (4) To further help grasp the similarities or differences exist among Manhattan hoods, I'll use the Folium library to illustrate the location of whatever clusters emerge from the algorithm.

## 2.0: Methodology

In [1]:
# Gathering the code collections needed for this notebook's code to work:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 2.1: Geospatial Data Gathering

#### 2.1.1: Geospatial Data Gathering for Manhattan

2.1.1.1: A file of geospatial data for every neighborhood in New York City was obtained from https://geo.nyu.edu/catalog/nyu_2451_34572.

In [2]:
# 2.1.1.1 Fulfimment code:
with open('newyork_data.json') as json_data:
    newyork_data_json_list = json.load(json_data)

2.1.1.2: The geospatial NYC neighborhood data was put into a Python pandas dataframe for eventual analysis.

In [3]:
# 2.1.1.2 Fulfillment code:

nyc_hood_data_list_column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']

nyc_hood_dataframe = pd.DataFrame(columns=nyc_hood_data_list_column_names)

nyc_hood_data_list = newyork_data_json_list['features']

for hood_data_bunch in nyc_hood_data_list:
    borough = hood_name = hood_data_bunch['properties']['borough'] 
    hood_name = hood_data_bunch['properties']['name']
        
    hood_lat_lon = hood_data_bunch['geometry']['coordinates']
    hood_lat = hood_lat_lon[1]
    hood_lon = hood_lat_lon[0]
    
    nyc_hood_dataframe = nyc_hood_dataframe.append({'Borough': borough,
                                          'Neighborhood': hood_name,
                                          'Latitude': hood_lat,
                                          'Longitude': hood_lon}, ignore_index=True)

2.1.1.3: A Manhattan-only neighborhood data set is made.

In [4]:
#2.1.1.3 Fulfillment code:
manhattan_hood_dataframe = nyc_hood_dataframe[nyc_hood_dataframe['Borough'] == 'Manhattan'].reset_index(drop=True)

In [22]:
manhattan_hood_dataframe.head(20)

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688
5,Manhattan,Manhattanville,40.816934,-73.957385
6,Manhattan,Central Harlem,40.815976,-73.943211
7,Manhattan,East Harlem,40.792249,-73.944182
8,Manhattan,Upper East Side,40.775639,-73.960508
9,Manhattan,Yorkville,40.775930,-73.947118


### 2.2: Foursquare Venu Data Gathering

#### To associate information about points of interests in Manhattan to their locations and neighborhoods, I gather data from Foursquare.

#### 2.2.1: I compose the following function to ask Foursquare for info about the 100 most popular venues within 500 meters of any neighborhood's "location."

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]

    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            '3EM4OAFXSIALQKYIZO0NHII14GHQAVFMR3BM4POPPAY2BX50', 
            '0ZJDMFVXRDDH3J3TPAJMJA1ZE2VL3JFC2K42A1LHPVTAXRSR', 
            '20180604', 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']    
        
    return(nearby_venues)

#### 2.2.2: Foursquare Venue Data Gathering for Manhattan Neighborhoods

I apply the above function to the Manhattan neighborhood data set to find the 100 most popular venues in each Manhattan neighborhood.

In [24]:
#2.2.2 Fulfillment code:

top_100_venues_for_each_manhattan_hood = getNearbyVenues(names=manhattan_hood_dataframe['Neighborhood'],
                                   latitudes=manhattan_hood_dataframe['Latitude'],
                                   longitudes=manhattan_hood_dataframe['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [25]:
top_100_venues_for_each_manhattan_hood.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Astral Fitness & Wellness Center,40.876705,-73.906372,Gym


### 2.3: Examining Similarities in the Popularity of Venues among Manhattan Neighborhoods 

#### The similarity among hoods in Manhattan will be estimated via the *k*-means clustering algorithm.

#### 2.3.1 One-Hot Encoding

To use the *k*-means clustering algorithm, how frequently certains kinds of venues pop up in a neighborhood must be calculated. This may be done by "one-hot encoding" the various venues in the data sets created in 2.2.2 and 2.2.3.
In each row, a single venue gets a one or a zero for a score under each of the different categories of venues Foursquare recognizes, according to whether the venue is or isn't in that category of venue.

2.3.1.1: One-Hot Encoding Manhattan venues:

In [26]:
# 2.3.1.1 Fulfillment code:

# One-hot encoding for Manhattan venues
manhattan_venue_category_onehot = pd.get_dummies(top_100_venues_for_each_manhattan_hood[['Venue Category']], prefix="", prefix_sep="")

# Adding the Neighborhood and Neighboorhood Latitude and Longitude columns to the one-hot dataframe created above
manhattan_venue_category_onehot['Neighborhood'] = top_100_venues_for_each_manhattan_hood['Neighborhood']
manhattan_venue_category_onehot['Neighborhood Latitude'] = top_100_venues_for_each_manhattan_hood['Neighborhood Latitude']
manhattan_venue_category_onehot['Neighborhood Longitude'] = top_100_venues_for_each_manhattan_hood['Neighborhood Longitude']


# Moving the columns just added from to become the leftmost three columns
adjusted_manhattan_venues_onehot_columns = list(manhattan_venue_category_onehot.columns[-3:]) + list(manhattan_venue_category_onehot.columns[:-3])
manhattan_venue_category_onehot_final = manhattan_venue_category_onehot[adjusted_manhattan_venues_onehot_columns]

In [28]:
manhattan_venue_category_onehot_final.head(25)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smo

In [14]:
adjusted_manhattan_venues_onehot_columns

[Index(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude'], dtype='object'),
 'American Restaurant',
 'Coffee Shop',
 'Deli / Bodega',
 'Department Store',
 'Diner',
 'Discount Store',
 'Donut Shop',
 'Gym',
 'Ice Cream Shop',
 'Kids Store',
 'Miscellaneous Shop',
 'Pharmacy',
 'Pizza Place',
 'Sandwich Place',
 'Seafood Restaurant',
 'Shopping Mall',
 'Steakhouse',
 'Supplement Shop',
 'Tennis Stadium',
 'Video Game Store',
 'Yoga Studio']

In [13]:
manhattan_venue_category_onehot.head()

,American Restaurant,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Gym,Ice Cream Shop,Kids Store,Miscellaneous Shop,Pharmacy,Pizza Place,Sandwich Place,Seafood Restaurant,Shopping Mall,Steakhouse,Supplement Shop,Tennis Stadium,Video Game Store,Yoga Studio,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,Marble Hill,40.876551,-73.91066
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Marble Hill,40.876551,-73.91066
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Marble Hill,40.876551,-73.91066
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Marble Hill,40.876551,-73.91066
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,Marble Hill,40.876551,-73.91066


#### 2.3.2 Categorizing the Popularity of a Venue Category for each Hood

The rows of the one-hot dataframes show whether a venue fits into different categories of venues. To know how the venue categories are distributed in each neighborhood--the relative frequency with which a certain category of venue shows up in a neighborhood--would be a useful characteristic of a hood to know. This relative frequency is sort of a probability that a single neighborhood venue will be a venue of a certain category. 

A Python pandas `groupby` function may help to calculate this relative frequency distribution--which is, again, in other words, the chances that in a certain neighborhood, a single random venue fits in specific different categories of venues. The 'groupby' function requires a calculation to apply to every venue in a neighborhood. A function that will help calculate the probability of a single random venue--or, the average venue--in a neighborhood being in different categories of venues is the `mean` function.

2.3.2.1: Grouping Manhattan Venues by Neighborhood, Calculating the Relative Frequency with which the "Average" Venue in a Neighborhood Fits Various Categories of Venues

In [29]:
# 2.3.2.1 Fulfillment code:
average_manhattan_venue_in_a_neighborhood_relative_frequencies_for_being_in_different_venue_categories = manhattan_venue_category_onehot_final.groupby('Neighborhood').mean().reset_index()

In [30]:
average_manhattan_venue_in_a_neighborhood_relative_frequencies_for_being_in_different_venue_categories.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smo

#### 2.3.4: K-means Clustering Algorithm Calculation

The alogrithm will divide the pool of Manhattan neighborhoods into two clusters. This should help us judge how much Manhattan is divided into two, if neighborhoods and areas of Manhattan have significantly different cultural environments as estimated by venue categories.

In [34]:
#2.3.4 Fulfillment code:
onehot_average_per_hood = average_manhattan_venue_in_a_neighborhood_relative_frequencies_for_being_in_different_venue_categories.drop(["Neighborhood","Neighborhood Latitude","Neighborhood Longitude"],1)

kclusters=2

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(onehot_average_per_hood)

#### 2.3.5: Calculating Tables Showing Manhattan Divided into Two *K*-means Clusters

In [35]:
# 2.3.5 Fulfillment Code:

# Labeling Manhattan hoods by the cluster they were assigned to by the k-means algorithm:
average_manhattan_venue_in_a_neighborhood_relative_frequencies_for_being_in_different_venue_categories['Cluster Assigned']=kmeans.labels_

# Splitting the two clusters into separate dataframes:
first_cluster_of_manhattan = average_manhattan_venue_in_a_neighborhood_relative_frequencies_for_being_in_different_venue_categories[average_manhattan_venue_in_a_neighborhood_relative_frequencies_for_being_in_different_venue_categories['Cluster Assigned']==0]

second_cluster_of_manhattan = average_manhattan_venue_in_a_neighborhood_relative_frequencies_for_being_in_different_venue_categories[average_manhattan_venue_in_a_neighborhood_relative_frequencies_for_being_in_different_venue_categories['Cluster Assigned']==1]

###  2.4 Visualizing the Potential Manhattan Divide

In [36]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [42]:
#conda install -c conda-forge folium=0.5.0 --yes
#import folium # map rendering library

# create map of Manhattan using latitude and longitude values
# map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map for first cluster
for lat, lng, label in zip(first_cluster_of_manhattan['Neighborhood Latitude'], first_cluster_of_manhattan['Neighborhood Longitude'], first_cluster_of_manhattan['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan) 

In [58]:
# add markers to map for second cluster
for lat, lng, label in zip(second_cluster_of_manhattan['Neighborhood Latitude'], second_cluster_of_manhattan['Neighborhood Longitude'], first_cluster_of_manhattan['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)      

## 3.0: Results

### 3.1: Hood Divide in Manhattan

How much do Manhattan hoods self-segregate, based on the *k*-means algorithm?

#### 3.1.1: Counting the number of different hoods in the first cluster:

In [54]:
len(first_cluster_of_manhattan['Neighborhood'].unique())

39

There are 39 different hoods in the first cluster, and ...

#### 3.1.2 Counting the number of different hoods in the second cluster:

In [55]:
len(second_cluster_of_manhattan['Neighborhood'].unique())

1

In [56]:
second_cluster_of_manhattan

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smo

There's 1 Manhattan hood in the second cluster.

### 3.2: Visualizing Hood Divide

Here's what a map of Manhattan venue diversity looks like:

In [59]:
map_manhattan

## 4.0: Discussion

### 4.1: Hood Similarities between Manhattan and Toronto

The numbers tell us there're 39 hoods in one "cluster" of Manhattan, 1 in the other.

### 4.2: Visualizing Hood Divide

It's pretty clear: Stuyvesant Town--what NYers call Stuy Town--is pretty divisive.

It's a weird place. It looks like "the projects"--urban dictionary might help you--and it *is* a housing project--just one where people making 100K+ live. It's exclusive but really nonsensical--all that money for sh**-brick looking buildings, massive shadows on the street, on the tiny lawns no one plays on and lonely little city trees in front of the 30 story buildings. It's depressing to look at.

I guess people could riot against the relatively rich there, but they couldn't bust anything up--the buildings aint fancy at all on the outside: just brick walls.


### 4.3: Further Possible Considerations

Of course, you could look at specific parts of the data set that went into the *k*-means clustering algorithm, for quick snapshots of the similarities of the hoods: 

you could look at the ten most popular venues in each nieghborhood, see how much they overlap between hoods; 

or

you could look at the sheer number of different categories of venues in hoods across Manhattan, for a guess at the 'econo-cultural diversity' of the locales.

## 5: Conclusion

Get the hell out of NYC--or vote Republican and reason with the sane remnant left there to do the same.

Just move to Florida, like rich Manhattanites tend to do.

Or stay and fight the incoherence with facts and logic. There's enough patronizing going around.

Good luck.